# Discovery of Reticular Materials for C02 Capture

## MOF Reticular Materials
Metal-Organic Frameworks (MOFs) are crystalline materials consisting of metal ions or clusters coordinated to organic ligands to form one-, two-, or three-dimensional structures. They have exceptional properties including:

- Ultra-high porosity
- Large surface areas (up to 7,000 m²/g)
- Tunable pore sizes and chemical functionalities

MOFs have applications in gas storage, separation, catalysis, sensing, and drug delivery. However, with millions of possible combinations of building blocks, finding MOFs with optimal properties for CO2 capture is challenging.

In [2]:
import warnings
warnings.filterwarnings("ignore")

# Import necessary libraries
import sys
import math
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import os
import nglview as nv
import tempfile
import subprocess

# Import framework components
from material.builder import MaterialBuilder
from material.masks import BaseMask, RMSDMask, AngleMask, CombinedMask

# Set plotting style
sns.set_theme()
plt.rcParams.update({'font.size': 12})

## 1. Understanding Masks for Building Block Selection

Masks are crucial components that determine which building blocks are compatible with each position in a topology. They filter the available options based on geometric and structural compatibility.

Three types of masks are implemented:

1. **RMSDMask**: Filters building blocks based on Root Mean Square Deviation (RMSD) between the building block and local structure in the topology
2. **AngleMask**: Filters based on angular deviations between connection points
3. **CombinedMask**: Uses both RMSD and angular criteria for more precise filtering

Let's explore how these different masks affect the MOF design process.

### 1.1 Loading a Topology

First, we need to define a topology that serves as the blueprint for our MOF design. We'll use the "ffc" topology, which is a primitive cubic structure commonly used in MOF design.

In [17]:
# Define the topology to use
topology_string = "tbo"

# Create a basic material builder with default RMSDMask
material_builder = MaterialBuilder(topology_string)

# Print some information about the topology
print(f"Topology: {topology_string}")
print(f"Number of unique nodes: {material_builder.n_nodes}")
print(f"Number of edge types: {material_builder.topology.n_edge_types}")
print(f"Total slots (nodes + edges): {material_builder.n_slots}")

Topology: tbo
Number of unique nodes: 2
Number of edge types: 1
Total slots (nodes + edges): 3


### 1.2 Comparing Different Masks

In [18]:
# Create different masks
rmsd_mask = RMSDMask(rmsd_cutoff=0.3)  # Default: more permissive
angle_mask = AngleMask(angle_cutoff=30.0)  # Filter by connection point angles
combined_mask = CombinedMask(rmsd_cutoff=0.3, angle_cutoff=30.0)  # More restrictive

# Create material builders with different masks
builder_rmsd = MaterialBuilder(topology_string, mask=rmsd_mask)
builder_angle = MaterialBuilder(topology_string, mask=angle_mask)
builder_combined = MaterialBuilder(topology_string, mask=combined_mask)

# Compare the number of allowed building blocks for each position
print("Number of allowed building blocks at each node position:")
for i in range(builder_rmsd.n_nodes):
    rmsd_allowed = len(builder_rmsd.mask.forward_actions_at_each_slot[i])
    angle_allowed = len(builder_angle.mask.forward_actions_at_each_slot[i])
    combined_allowed = len(builder_combined.mask.forward_actions_at_each_slot[i])
    print(f"Position {i}: RMSD={rmsd_allowed}, Angle={angle_allowed}, Combined={combined_allowed}")

Number of allowed building blocks at each node position:
Position 0: RMSD=73, Angle=73, Combined=73
Position 1: RMSD=102, Angle=22, Combined=19


In [19]:
# Total Possible building blocks can be constructed with above constraints
rmsd_allowed = math.prod([len(x) for x in builder_rmsd.mask.forward_actions_at_each_slot])
angle_allowed = math.prod([len(x) for x in builder_angle.mask.forward_actions_at_each_slot])
combined_allowed = math.prod([len(x) for x in builder_combined.mask.forward_actions_at_each_slot])
print(f"Total possible building blocks")
print(f"RMSD: {rmsd_allowed:,} Angle: {angle_allowed:,} Combined: {combined_allowed:,}")

Total possible building blocks
RMSD: 1,638,120 Angle: 353,320 Combined: 305,140


### 1.3 Visualizing Structures

In [31]:
seq = builder_combined.random_sequence()
print(f"Random sequence: {seq}")

structure = builder_combined.make_structure(seq)
nv.show_pymatgen(structure)

Random sequence: ['N488', 'N417', 'E181']


NGLWidget()

**Theoretical Insight: Choosing the Right Mask**

The choice of mask involves a trade-off between:

- **Exploration space**: More permissive masks (like basic RMSD) allow exploration of a larger design space
- **Structural integrity**: More restrictive masks (like Combined) lead to more structurally sound MOFs

**Best Uses:**
- **RMSDMask** -> To explore a wide variety of potential structures
- **AngleMask** -> when connection geometries are critical for application (H<sub>2</sub>O seperation fromAir)
- **CombinedMask** -> when we need high-quality, structurally sound MOFs

For CO2 capture applications, the CombinedMask typically provides the best balance because structural integrity is crucial for maintaining adsorption sites. But trying two masks and compare
- Using Masks -> RMSDMask, CombinedMask

## 2. Reward Functions for CO2 Capture

To guide the MOF design process for CO2 capture, we need reward functions that quantify how well a MOF performs for this specific application. Key properties for CO2 capture include:

- **Surface area**: Larger surface areas generally provide more adsorption sites for CO2 molecules
- **Pore size distribution**: Pores of appropriate size can selectively capture CO2 over other gases
- **Chemical functionality**: Specific functional groups that interact strongly with CO2

Implementing Surface Area Rewarder

In [4]:
class SurfaceAreaRewarder():
    def __init__(self, builder, cutoff, temporary_dir = "scratch"):
        self.builder = builder
        self.cutoff = cutoff
        self.temporary_dir = temporary_dir
        os.makedirs(temporary_dir, exist_ok=True)

    def reward(self, sequence):
        with tempfile.NamedTemporaryFile(suffix=".cif", dir=self.temporary_dir, delete=True) as temp:
            mof = self.builder.make_pormake_mof(sequence)
            mof.write_cif(temp.name)

            command=(['./network']  + ['-sa'] + ['1.525'] + ['1.525'] + ['2000'] + [temp.name])
            subprocess.run(command,stdout = subprocess.DEVNULL)

            output_filename = temp.name.removesuffix(".cif") + ".sa"
            
            if os.path.exists(output_filename) == False:
                output_reward = 0
            else: 
                with open(output_filename) as result_file:
                    lines=[] 
                    for line in result_file:
                        lines.append(line.rstrip())

                    # Zeo++ writes empty output file for sone non-physical MOFs. Reward them as 0.
                    if len(lines)==0:
                        output_reward = 0
                    else:
                        frags=lines[0].split()
                        NASA=float(frags[17])
                        ASA=float(frags[11])

                        # Gravimetric surface area is NASA + ASA
                        area = ASA + NASA

                        output_reward = math.exp((area - self.cutoff) / self.cutoff)
            
                # Remove zeo++ output file
                # Temp file will be removed by the surrounding "with"
                os.remove(output_filename)            

            return output_reward

In [5]:
rewarder_combined = SurfaceAreaRewarder(builder_combined, 5000)
rewarder_rmsd = SurfaceAreaRewarder(builder_rmsd, 5000)

In [12]:
seq = builder_combined.random_sequence()
print(seq, rewarder_combined.reward(seq))

['N614', 'N201', 'N408', 'E8', 'E20'] 1.894244351678183


In [7]:
seq = builder_rmsd.random_sequence()
print(seq, rewarder_rmsd.reward(seq))

['N512', 'N387', 'N411', 'E189', 'E162'] 1.1690225823873872


## 3. 

In [6]:
from gflow import LSTM, SequenceEnvironment, TrajectoryBalanceGFlowNet
import torch

device = torch.device('cpu')

In [7]:
env_rmsd = SequenceEnvironment(
    token_vocabulary = builder_rmsd.token_vocabulary, 
    termination_token = builder_rmsd.termination_token,
    reward_function = lambda s: rewarder_rmsd.reward(s),
    render_function = None,
    mask = builder_rmsd.mask,
    max_sequence_length = builder_rmsd.n_slots, min_sequence_length = builder_rmsd.n_slots
    )

model_rmsd = LSTM(token_vocabulary = builder_rmsd.token_vocabulary, n_actions = env_rmsd.action_space.n).to(device)
agent_rmsd = TrajectoryBalanceGFlowNet(env_rmsd, model_rmsd).to(device)

In [9]:
agent_rmsd.train(True)
observations, infos, rewards, losses, logZs = agent_rmsd.fit(learning_rate=5e-3, num_episodes=200, minibatch_size=5)

2688.259, 1.1946440935134888: 100%|██████████| 200/200 [03:16<00:00,  1.02it/s]
